In [1]:
import flwr as fl
import tensorflow as tf
from tensorflow import keras
from keras import layers
import utils as ut
from typing import Dict, Optional, Tuple
from pathlib import Path
import numpy as np

In [2]:

def get_evaluate_fn(model, x_test, y_test):
    """Return an evaluation function for server-side evaluation."""

    # The `evaluate` function will be called after every round
    def evaluate(
        server_round: int,
        parameters: fl.common.NDArrays,
        config: Dict[str, fl.common.Scalar],
    ) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:

        # Update the model with the latest parameters
        model.set_weights(parameters)

        # Evaluate the model on the test data
        predictions = model.predict(x_test)
        loss = tf.keras.losses.mean_squared_error(y_test, predictions).numpy().mean().item()
        mae = tf.keras.metrics.mean_absolute_error(y_test, predictions).numpy().mean().item()

        return loss, {"mae": mae}

    return evaluate

def fit_config(server_round: int):
    """Return training configuration dict for each round.

    Keep batch size fixed at 32, perform two rounds of training with one local epoch,
    increase to two local epochs afterwards.
    """
    config = {
        "batch_size": 128,
        "local_epochs": 50 if server_round < 2 else 100,
    }
    return config


def evaluate_config(server_round: int):
    """Return evaluation configuration dict for each round.

    Perform five local evaluation steps on each client (i.e., use five batches) during
    rounds one to three, then increase to ten local evaluation steps.
    """
    val_steps = 5 if server_round < 4 else 10
    return {"val_steps": val_steps}

In [3]:
(x_train, y_train), (x_test, y_test) = ut.partition_dataset(2,3,1)

model = tf.keras.models.Sequential(
    [
        layers.Input(shape=(8,)),  
        layers.Dense(64, activation='relu'),
        layers.Dense(64, activation='relu'),  
        layers.Dense(1)  
    ]
)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', 'mse'])

strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.75,
    fraction_evaluate=1,
    min_fit_clients=3,
    min_evaluate_clients=4,
    min_available_clients=4,
    evaluate_fn=get_evaluate_fn(model,x_test,y_test),
    on_fit_config_fn=fit_config,
    on_evaluate_config_fn=evaluate_config,
    initial_parameters=fl.common.ndarrays_to_parameters(model.get_weights()),
)




In [4]:
# Start Flower server
history_with_noise = fl.server.start_server(
    server_address="127.0.0.1:8080",
    config=fl.server.ServerConfig(num_rounds=10),
    strategy = strategy
)

INFO flwr 2023-11-23 01:50:57,866 | app.py:162 | Starting Flower server, config: ServerConfig(num_rounds=10, round_timeout=None)
INFO flwr 2023-11-23 01:50:57,905 | app.py:175 | Flower ECE: gRPC server running (10 rounds), SSL is disabled
INFO flwr 2023-11-23 01:50:57,905 | server.py:89 | Initializing global parameters
INFO flwr 2023-11-23 01:50:57,906 | server.py:272 | Using initial parameters provided by strategy
INFO flwr 2023-11-23 01:50:57,906 | server.py:91 | Evaluating initial parameters


129/129 [==============================] - 0s 841us/step


INFO flwr 2023-11-23 01:50:58,285 | server.py:94 | initial parameters (loss, other metrics): 5.775308132171631, {'mae': 2.1126232147216797}
INFO flwr 2023-11-23 01:50:58,286 | server.py:104 | FL starting
DEBUG flwr 2023-11-23 01:51:37,756 | server.py:222 | fit_round 1: strategy sampled 3 clients (out of 4)
DEBUG flwr 2023-11-23 01:51:43,640 | server.py:236 | fit_round 1 received 3 results and 0 failures
WARNING flwr 2023-11-23 01:51:43,643 | fedavg.py:242 | No fit_metrics_aggregation_fn provided


129/129 [==============================] - 0s 937us/step


INFO flwr 2023-11-23 01:51:43,921 | server.py:125 | fit progress: (1, 2.1078832149505615, {'mae': 1.1092127561569214}, 45.63434220000636)
DEBUG flwr 2023-11-23 01:51:43,922 | server.py:173 | evaluate_round 1: strategy sampled 4 clients (out of 4)
DEBUG flwr 2023-11-23 01:51:44,771 | server.py:187 | evaluate_round 1 received 4 results and 0 failures
WARNING flwr 2023-11-23 01:51:44,773 | fedavg.py:273 | No evaluate_metrics_aggregation_fn provided
DEBUG flwr 2023-11-23 01:51:44,773 | server.py:222 | fit_round 2: strategy sampled 3 clients (out of 4)
DEBUG flwr 2023-11-23 01:51:55,186 | server.py:236 | fit_round 2 received 3 results and 0 failures


...

INFO flwr 2023-11-23 01:53:15,425 | server.py:125 | fit progress: (8, 2.3939924240112305, {'mae': 1.2009563446044922}, 137.1391534000286)
DEBUG flwr 2023-11-23 01:53:15,426 | server.py:173 | evaluate_round 8: strategy sampled 4 clients (out of 4)
DEBUG flwr 2023-11-23 01:53:16,439 | server.py:187 | evaluate_round 8 received 4 results and 0 failures
DEBUG flwr 2023-11-23 01:53:16,441 | server.py:222 | fit_round 9: strategy sampled 3 clients (out of 4)
DEBUG flwr 2023-11-23 01:53:30,222 | server.py:236 | fit_round 9 received 3 results and 0 failures


129/129 [==============================] - 0s 1ms/step


INFO flwr 2023-11-23 01:53:30,550 | server.py:125 | fit progress: (9, 2.3399317264556885, {'mae': 1.185317039489746}, 152.26293730002362)
DEBUG flwr 2023-11-23 01:53:30,551 | server.py:173 | evaluate_round 9: strategy sampled 4 clients (out of 4)
DEBUG flwr 2023-11-23 01:53:31,645 | server.py:187 | evaluate_round 9 received 4 results and 0 failures
DEBUG flwr 2023-11-23 01:53:31,646 | server.py:222 | fit_round 10: strategy sampled 3 clients (out of 4)
DEBUG flwr 2023-11-23 01:53:45,107 | server.py:236 | fit_round 10 received 3 results and 0 failures


129/129 [==============================] - 0s 991us/step


INFO flwr 2023-11-23 01:53:45,408 | server.py:125 | fit progress: (10, 2.357820749282837, {'mae': 1.1831157207489014}, 167.12189000000944)
DEBUG flwr 2023-11-23 01:53:45,409 | server.py:173 | evaluate_round 10: strategy sampled 4 clients (out of 4)
DEBUG flwr 2023-11-23 01:53:46,374 | server.py:187 | evaluate_round 10 received 4 results and 0 failures
INFO flwr 2023-11-23 01:53:46,375 | server.py:153 | FL finished in 168.0886041999911
INFO flwr 2023-11-23 01:53:46,376 | app.py:225 | app_fit: losses_distributed [(1, 2.127435028553009), (2, 2.389154374599457), (3, 2.4064800143241882), (4, 2.4260382652282715), (5, 2.7680426836013794), (6, 2.566336512565613), (7, 2.4655032753944397), (8, 2.995591878890991), (9, 2.5242376923561096), (10, 2.6346829533576965)]
INFO flwr 2023-11-23 01:53:46,377 | app.py:226 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-11-23 01:53:46,377 | app.py:227 | app_fit: metrics_distributed {}
INFO flwr 2023-11-23 01:53:46,377 | app.py:228 | app_fit: losses_centr

In [5]:
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.75,
    fraction_evaluate=1,
    min_fit_clients=3,
    min_evaluate_clients=4,
    min_available_clients=4,
    evaluate_fn=get_evaluate_fn(model,x_test,y_test),
    on_fit_config_fn=fit_config,
    on_evaluate_config_fn=evaluate_config,
    initial_parameters=fl.common.ndarrays_to_parameters(model.get_weights()),
)

history_without_noise = fl.server.start_server(
    server_address="127.0.0.1:8081",
    config=fl.server.ServerConfig(num_rounds=10),
    strategy = strategy
)

INFO flwr 2023-11-23 01:54:42,018 | app.py:162 | Starting Flower server, config: ServerConfig(num_rounds=10, round_timeout=None)


INFO flwr 2023-11-23 01:54:42,048 | app.py:175 | Flower ECE: gRPC server running (10 rounds), SSL is disabled
INFO flwr 2023-11-23 01:54:42,049 | server.py:89 | Initializing global parameters
INFO flwr 2023-11-23 01:54:42,050 | server.py:272 | Using initial parameters provided by strategy
INFO flwr 2023-11-23 01:54:42,050 | server.py:91 | Evaluating initial parameters


129/129 [==============================] - 0s 875us/step


INFO flwr 2023-11-23 01:54:42,304 | server.py:94 | initial parameters (loss, other metrics): 2.357820749282837, {'mae': 1.1831157207489014}
INFO flwr 2023-11-23 01:54:42,305 | server.py:104 | FL starting
DEBUG flwr 2023-11-23 01:55:11,201 | server.py:222 | fit_round 1: strategy sampled 3 clients (out of 4)
DEBUG flwr 2023-11-23 01:55:17,412 | server.py:236 | fit_round 1 received 3 results and 0 failures
WARNING flwr 2023-11-23 01:55:17,414 | fedavg.py:242 | No fit_metrics_aggregation_fn provided


129/129 [==============================] - 0s 852us/step


...

INFO flwr 2023-11-23 01:56:48,958 | server.py:125 | fit progress: (8, 2.445713520050049, {'mae': 1.188620924949646}, 126.65209210000467)
DEBUG flwr 2023-11-23 01:56:48,959 | server.py:173 | evaluate_round 8: strategy sampled 4 clients (out of 4)
DEBUG flwr 2023-11-23 01:56:49,909 | server.py:187 | evaluate_round 8 received 4 results and 0 failures
DEBUG flwr 2023-11-23 01:56:49,910 | server.py:222 | fit_round 9: strategy sampled 3 clients (out of 4)
DEBUG flwr 2023-11-23 01:57:02,120 | server.py:236 | fit_round 9 received 3 results and 0 failures


129/129 [==============================] - 0s 1ms/step


INFO flwr 2023-11-23 01:57:02,420 | server.py:125 | fit progress: (9, 2.4645471572875977, {'mae': 1.191007375717163}, 140.1144841000205)
DEBUG flwr 2023-11-23 01:57:02,421 | server.py:173 | evaluate_round 9: strategy sampled 4 clients (out of 4)
DEBUG flwr 2023-11-23 01:57:03,384 | server.py:187 | evaluate_round 9 received 4 results and 0 failures
DEBUG flwr 2023-11-23 01:57:03,386 | server.py:222 | fit_round 10: strategy sampled 3 clients (out of 4)
DEBUG flwr 2023-11-23 01:57:15,416 | server.py:236 | fit_round 10 received 3 results and 0 failures


129/129 [==============================] - 0s 1ms/step


INFO flwr 2023-11-23 01:57:15,732 | server.py:125 | fit progress: (10, 2.377683639526367, {'mae': 1.1676628589630127}, 153.42621270002564)
DEBUG flwr 2023-11-23 01:57:15,733 | server.py:173 | evaluate_round 10: strategy sampled 4 clients (out of 4)
DEBUG flwr 2023-11-23 01:57:16,650 | server.py:187 | evaluate_round 10 received 4 results and 0 failures
INFO flwr 2023-11-23 01:57:16,651 | server.py:153 | FL finished in 154.34498789999634
INFO flwr 2023-11-23 01:57:16,651 | app.py:225 | app_fit: losses_distributed [(1, 2.2793779373168945), (2, 2.3680520057678223), (3, 2.3373703956604004), (4, 2.2790939807891846), (5, 2.282250165939331), (6, 2.2932262420654297), (7, 2.2717788219451904), (8, 2.399872064590454), (9, 2.4535892009735107), (10, 2.3278539180755615)]
INFO flwr 2023-11-23 01:57:16,652 | app.py:226 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-11-23 01:57:16,652 | app.py:227 | app_fit: metrics_distributed {}
INFO flwr 2023-11-23 01:57:16,652 | app.py:228 | app_fit: losses_ce

In [6]:
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.75,
    fraction_evaluate=1,
    min_fit_clients=3,
    min_evaluate_clients=4,
    min_available_clients=4,
    evaluate_fn=get_evaluate_fn(model,x_test,y_test),
    on_fit_config_fn=fit_config,
    on_evaluate_config_fn=evaluate_config,
    initial_parameters=fl.common.ndarrays_to_parameters(model.get_weights()),
)

history_with_minus_noise = fl.server.start_server(
    server_address="127.0.0.1:8082",
    config=fl.server.ServerConfig(num_rounds=10),
    strategy = strategy
)

INFO flwr 2023-11-23 01:59:42,216 | app.py:162 | Starting Flower server, config: ServerConfig(num_rounds=10, round_timeout=None)
INFO flwr 2023-11-23 01:59:42,234 | app.py:175 | Flower ECE: gRPC server running (10 rounds), SSL is disabled
INFO flwr 2023-11-23 01:59:42,234 | server.py:89 | Initializing global parameters
INFO flwr 2023-11-23 01:59:42,235 | server.py:272 | Using initial parameters provided by strategy
INFO flwr 2023-11-23 01:59:42,235 | server.py:91 | Evaluating initial parameters


129/129 [==============================] - 0s 953us/step


INFO flwr 2023-11-23 01:59:42,508 | server.py:94 | initial parameters (loss, other metrics): 2.377683639526367, {'mae': 1.1676628589630127}
INFO flwr 2023-11-23 01:59:42,509 | server.py:104 | FL starting
DEBUG flwr 2023-11-23 02:00:05,392 | server.py:222 | fit_round 1: strategy sampled 3 clients (out of 4)
DEBUG flwr 2023-11-23 02:00:11,341 | server.py:236 | fit_round 1 received 3 results and 0 failures
WARNING flwr 2023-11-23 02:00:11,343 | fedavg.py:242 | No fit_metrics_aggregation_fn provided


129/129 [==============================] - 0s 757us/step


INFO flwr 2023-11-23 02:00:11,575 | server.py:125 | fit progress: (1, 2.7696542739868164, {'mae': 1.2578725814819336}, 29.065098599996418)
DEBUG flwr 2023-11-23 02:00:11,576 | server.py:173 | evaluate_round 1: strategy sampled 4 clients (out of 4)
DEBUG flwr 2023-11-23 02:00:12,497 | server.py:187 | evaluate_round 1 received 4 results and 0 failures
WARNING flwr 2023-11-23 02:00:12,498 | fedavg.py:273 | No evaluate_metrics_aggregation_fn provided
DEBUG flwr 2023-11-23 02:00:12,499 | server.py:222 | fit_round 2: strategy sampled 3 clients (out of 4)
DEBUG flwr 2023-11-23 02:00:22,674 | server.py:236 | fit_round 2 received 3 results and 0 failures


...

INFO flwr 2023-11-23 02:01:43,977 | server.py:125 | fit progress: (8, 2.828364133834839, {'mae': 1.2622789144515991}, 121.46677659999114)
DEBUG flwr 2023-11-23 02:01:43,978 | server.py:173 | evaluate_round 8: strategy sampled 4 clients (out of 4)
DEBUG flwr 2023-11-23 02:01:44,977 | server.py:187 | evaluate_round 8 received 4 results and 0 failures
DEBUG flwr 2023-11-23 02:01:44,978 | server.py:222 | fit_round 9: strategy sampled 3 clients (out of 4)
DEBUG flwr 2023-11-23 02:01:57,445 | server.py:236 | fit_round 9 received 3 results and 0 failures


129/129 [==============================] - 0s 1ms/step


INFO flwr 2023-11-23 02:01:57,819 | server.py:125 | fit progress: (9, 2.8816874027252197, {'mae': 1.2733802795410156}, 135.30840239999816)
DEBUG flwr 2023-11-23 02:01:57,819 | server.py:173 | evaluate_round 9: strategy sampled 4 clients (out of 4)
DEBUG flwr 2023-11-23 02:01:58,730 | server.py:187 | evaluate_round 9 received 4 results and 0 failures
DEBUG flwr 2023-11-23 02:01:58,731 | server.py:222 | fit_round 10: strategy sampled 3 clients (out of 4)
DEBUG flwr 2023-11-23 02:02:11,211 | server.py:236 | fit_round 10 received 3 results and 0 failures


129/129 [==============================] - 0s 1ms/step


INFO flwr 2023-11-23 02:02:11,590 | server.py:125 | fit progress: (10, 2.8611953258514404, {'mae': 1.2631112337112427}, 149.07998789998237)
DEBUG flwr 2023-11-23 02:02:11,591 | server.py:173 | evaluate_round 10: strategy sampled 4 clients (out of 4)
DEBUG flwr 2023-11-23 02:02:12,499 | server.py:187 | evaluate_round 10 received 4 results and 0 failures
INFO flwr 2023-11-23 02:02:12,500 | server.py:153 | FL finished in 149.9902371999342
INFO flwr 2023-11-23 02:02:12,501 | app.py:225 | app_fit: losses_distributed [(1, 2.3105416297912598), (2, 2.3101747035980225), (3, 2.267969846725464), (4, 2.3874034881591797), (5, 2.3560914993286133), (6, 2.4283580780029297), (7, 2.404742479324341), (8, 2.398416757583618), (9, 2.399165391921997), (10, 2.4310128688812256)]
INFO flwr 2023-11-23 02:02:12,502 | app.py:226 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-11-23 02:02:12,502 | app.py:227 | app_fit: metrics_distributed {}
INFO flwr 2023-11-23 02:02:12,503 | app.py:228 | app_fit: losses_cent

In [ ]:
print(history)

History (loss, distributed):
	round 1: 2.647073984146118
	round 2: 1.4041863679885864
	round 3: 1.6445988416671753
	round 4: 1.935247778892517
	round 5: 2.0753211975097656
	round 6: 2.0844905376434326
	round 7: 2.086702585220337
	round 8: 2.0768063068389893
	round 9: 2.1068055629730225
	round 10: 2.0973358154296875
History (loss, centralized):
	round 0: 5.924674987792969
	round 1: 2.647073984146118
	round 2: 1.4041863679885864
	round 3: 1.6445988416671753
	round 4: 1.935247778892517
	round 5: 2.0753211975097656
	round 6: 2.0844905376434326
	round 7: 2.086702585220337
	round 8: 2.0768063068389893
	round 9: 2.1068055629730225
	round 10: 2.0973358154296875
History (metrics, centralized):
{'mae': [(0, 2.1480019092559814), (1, 1.211744785308838), (2, 0.9283948540687561), (3, 1.0063337087631226), (4, 1.0710625648498535), (5, 1.1068549156188965), (6, 1.107938289642334), (7, 1.1093168258666992), (8, 1.1095515489578247), (9, 1.1134757995605469), (10, 1.115409255027771)]}
